In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("EmotionCSV.csv")
#df2 = pd.read_csv("./Training_Dataframes/esken30ktweetswithemotions.csv")

In [2]:
#print(type(df))
#print(df.head())
df.drop(["Unnamed: 0"],axis = 1, inplace = True)
#print("----")

In [3]:
#print(df.head())
#print(df.keys())
keyString = df.keys()
keyString += "Input"
df.columns = keyString
#print(df.keys())
print(keyString[0:24])

Index(['tweetidInput', 'datetimeInput', 'dateInput', 'timeInput', 'userInput',
       'textInput', 'languageInput', 'editedInput', 'textforttbInput',
       'lematizedInput', 'AnalysisInput', 'word_countInput', 'WutInput',
       'VorfreudeInput', 'EkelInput', 'FurchtInput', 'FreudeInput',
       'TraurigkeitInput', 'UeberraschungInput', 'VertrauenInput',
       'neg_emotionsInput', 'pos_emotionsInput', 'total_neg_emotionsInput',
       'total_pos_emotionsInput'],
      dtype='object')


In [4]:
listOfTweets = df["editedInput"].tolist()
docs = np.array(listOfTweets)

In [5]:
# https://medium.com/swlh/tweet-sentiment-analysis-using-python-for-complete-beginners-4aeb4456040
# Turning all Tweets into Vectors for later Machine learning
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(binary = True)
oneHotCv = cv.fit_transform(docs)
oneHotArray = oneHotCv.toarray()
oneHotArray.shape
oneHotDf = pd.DataFrame(oneHotArray)
#Naming the Columns accordingly for humans to read
oneHotDf.columns = cv.get_feature_names_out()
#print(oneHotDf.head())
#print(oneHotDf.info())
longDf = pd.concat([df,oneHotDf], axis = 1)
#dir(CountVectorizer)
print(longDf.describe())
longDf.head()

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names_out'

In [ ]:
#proving, that all words in the tweet are marked appropriatly
print(longDf.iloc[0]["editedInput"])
print(longDf.iloc[0][longDf.iloc[0]["editedInput"].split()])

In [ ]:
print(longDf.keys()[:27])
print(longDf.keys()[:24])
#Shows that keys[0:24] are tweet data, thats not part of the NLP

In [ ]:
#This is already part of old code
import re

def clean(text):
    text = re.sub(r'@[A-Za-z0-9]+\s?', '', text) #Removed Mentions
    text = re.sub(r'#', '', text) #Removed #
    text = re.sub(r'(.)1+', r'1', text) #cleaned single letters
    text = re.sub('((www.[^s]+)|(https?://[^s]+))','',text) #Removes links
    text = re.sub('@','',text) #Remove @
    text = re.sub('-','',text) #Remove -
    text = re.sub('ä','ae',text) #Remove ä
    text = re.sub('Ä','Ae',text) #Remove Ä
    text = re.sub('ö','oe',text) #Remove Ä
    text = re.sub('Ö','Oe',text) #Remove Ä
    text = re.sub('ü','ue',text) #Remove Ä
    text = re.sub('Ü','Ue',text) #Remove Ä
    return text

import nltk
from nltk.stem import PorterStemmer
porter = PorterStemmer()
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data

In [ ]:
def sentence_toVec(sentence,goalDF):
    #Preparing Input text
    cleanSentence = sentence.lower()
    cleanSentence = clean(cleanSentence)
    cleansentene = stemming_on_text(cleanSentence)
    
    #Preparing EmptyDF
    emptyList = [0]*len(goalDF.keys())
    #print(emptyList)
    #print("Len EmptyList: " + str(len(emptyList)))
    #emptyDF = goalDF.iloc[0:0].copy()
    #emptyDF = emptyDF.append(emptyList)
    #emptyDF = emptyDF.append(pd.DataFrame(emptyList, columns = goalDF.keys()), ignore_index = True)
    emptyDF = pd.DataFrame(columns = goalDF.keys())
    #emptyDF.append(pd.Series(), ignore_index = True)
    emptyDF.loc[len(emptyDF)] = emptyList
    #print(emptyDF)
    #emptyDF = pd.DataFrame(emptyList, columns = goalDF.keys())
    #print("LenEmpty DF: " + str(len(emptyDF)) + " Len Keys Empty DF: " + str(len(emptyDF.keys())))
    
    emptyDF["textInput"][0] = sentence
    emptyDF["editedInput"][0] = cleanSentence
    #keys = list(emptyDF.keys()[0:10])
    #print("---")
    #print("New LenEmpty DF: " + str(len(emptyDF)) + " Len Keys Empty DF: " + str(len(emptyDF.keys())))
    #print(emptyDF["textInput"].tolist())
    listOfWords = cleanSentence.split()
    for word in listOfWords:
        if word in emptyDF.columns:
            emptyDF[word][0] = 1
    emptyDF.fillna(0)
    return emptyDF

In [ ]:
myPrivateSentence = "Wir sind falsch"    
print(longDf.keys()[24])

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
#print(list((keys)))
def trainEmotionModels(dataFrame):
    tweet_features = longDf.keys()[24:]
    X_trainingData = longDf[tweet_features]
    keys = list(longDf.keys()[12:20])
    listOfModels = []
    iterator = 0
    for emotion in longDf[keys]:
        print(emotion[:-5] + ": ")
        #print(longDf[emotion].head())
        y_emotion = longDf[emotion]
        train_X, val_X, train_y, val_y = train_test_split(X_trainingData, y_emotion,test_size = 0.33, random_state = 1,shuffle=True)
        emotion_Model = DecisionTreeRegressor(random_state = 1)
        emotion_Model.fit(X_trainingData,y_emotion)
        val_predictions = emotion_Model.predict(val_X)
        listOfModels.append(emotion_Model)
        #print(emotion_Model.predict(val_X.head()))
        
        print("Mean Absolute error for {}: ".format(emotion[:-5]) + str(round(mean_absolute_error(val_y,val_predictions)*100,2))+"%\n")
        #print("type(train_X): " + str(type(train_X)))
        #print("type(dfSentence): " + str(type(dfOfSentence)))
        #print("Predicition for for train_X.head() for emotion {}: ".format(emotion[:-5])+ str(emotion_Model.predict(train_X.head())))
        #print("Predicition for '{}' for emotion {}: ".format(myPrivateSentence,emotion[:-5])+ str(emotion_Model.predict(dfOfSentence[tweet_features])))
        #y_emotionGoal = longDf[emotion]
        
        
    dicOfModels = {"Wut-Model":listOfModels[0],"Vorfreude-Model":listOfModels[1],"Ekel-Model":listOfModels[2],"Furcht-Model":listOfModels[3],"Freude-Model":listOfModels[4],"Traurigkeit-Model":listOfModels[5], "Ueberraschungs-Model":listOfModels[6],"Vetrauen-Model":listOfModels[7]}
    return dicOfModels

In [ ]:
def getAttributesOfTweet(tweetNumber, df):
    keys = list(df.keys()[12:20])
    print("---\nTweet \n--- \n{}\n--- \nhas these Emotions according to DataInput Tweet:")
    for emotion in df[keys]:
        print(str(emotion[:-5]) + "-Value: " +str(df[emotion][tweetNumber]))

In [ ]:
def interpretOwnSentence(sentence, dicOfModels, df):
    if(type(sentence) != str):
        print("Sentence-Parameter is not a String")
        return
    if(type(dicOfModels) != dict):
        print("dicOfModels-Parameter is not a dictionary")
        return
    if(len(dicOfModels)!=8):
        print("dicOfModels Length is not 8")
        return
        
    #Transforming Sentence into Vector
    dfOfSentence = sentence_toVec(myPrivateSentence, df)
    #print("----\nThis is the shape of the df: \n" + str(dfOfSentence) + "\n----")
    listOfIndex = dfOfSentence.columns[(dfOfSentence == 1).all()].tolist()
    listOfIndex.extend(["textInput","editedInput","WutInput"])
    #print("----\nInput Sentence was transformed to this vector: \n" +str(dfOfSentence.head()))
    
    
    keys = list(df.keys()[12:20])
    #Applying different Models to sentence
    keyOfFeatures = dfOfSentence.keys()[24:]
    features = dfOfSentence[keyOfFeatures]
    iterator = 0
    for model in dicOfModels:
        #print(str(model) + " is the following type: " + str(type(model)))
        result = dicOfModels[model].predict(features)
        #print("---\nThis is the result of {}: ".format(str(model)) + str(result))
        #print("result[0]: " + str(result[0]))
        print("before writing: " + str(dfOfSentence[dfOfSentence.keys()[12+iterator]]))
        dfOfSentence[12+iterator] = result[0]
        print("---\nThis is in the dfOfSentence: " + str(dfOfSentence[12+iterator][0]))
        iterator = iterator + 1
    print("---\nThis is what all the Attributes look like: \n" + str(dfOfSentence[dfOfSentence.keys()[12:20]]))
    getAttributesOfTweet(0,dfOfSentence)

In [ ]:
giveMeMyModels = trainEmotionModels(longDf)

In [ ]:
interpretOwnSentence("Wir sind falsch", giveMeMyModels, longDf)